# 4. Reflection Pattern with Semantic Kernel

The reflection pattern involves an agent generating content and then having another agent (or the same agent) reflect on that content to verify it meets certain criteria. If the criteria aren't met, the agent refines the content. This pattern is useful for ensuring quality and completeness of responses.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents import ChatCompletionAgent
from typing import Annotated, List, Dict, Any
import json

Set up the kernel and service

In [ ]:
# Create a kernel instance
kernel = Kernel()

# Add Azure OpenAI chat completion service
service_id = "azure_openai"
kernel.add_service(
    AzureChatCompletion(
        service_id=service_id,
        deployment_name="gpt-4.1-mini",
    )
)

Create the travel advice agent

In [ ]:
# Travel Advisor Agent
travel_agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=kernel,
    name="TravelAdvisor",
    instructions="""You are a helpful assistant that gives comprehensive travel advice based on the user's input.
    
Provide detailed information about destinations including:
- Sports activities and facilities available
- Cultural attractions and experiences
- Historical sites and significance

Always strive to include information about all three aspects when possible.""",
    execution_settings={
        service_id: kernel.get_prompt_execution_settings_from_service_id(service_id)
    }
)

Create the reflection agent

In [ ]:
# Reflection Agent
reflection_agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=kernel,
    name="ContentReflector",
    instructions="""You are a content expert that validates travel advice for completeness.
    
Analyze the provided travel advice and check if it contains information about these topics:
- sports: Sports activities, facilities, recreational activities
- cultural: Cultural attractions, local customs, arts, festivals
- historical: Historical sites, monuments, historical significance

Respond with ONLY a JSON object in this exact format:
{
  "sports": {"present": true/false, "details": "explanation"},
  "cultural": {"present": true/false, "details": "explanation"},
  "historical": {"present": true/false, "details": "explanation"},
  "missing_topics": ["list of missing topic names"]
}

Do not include any other text or formatting.""",
    execution_settings={
        service_id: kernel.get_prompt_execution_settings_from_service_id(service_id)
    }
)

Create reflection workflow functions

In [ ]:
async def get_travel_advice(user_query: str, additional_requirements: str = "") -> str:
    """
    Get travel advice from the travel agent
    """
    print("→ Generating travel advice...")
    
    chat_history = ChatHistory()
    
    # Add the original user query
    full_query = user_query
    if additional_requirements:
        full_query += f"\n\nAdditional requirements: {additional_requirements}"
    
    chat_history.add_user_message(full_query)
    
    async for response in travel_agent.invoke(chat_history):
        return str(response.content)
    
    return "Unable to generate travel advice."

async def reflect_on_content(content: str) -> Dict[str, Any]:
    """
    Reflect on the travel advice content to check completeness
    """
    print("→ Reflecting on content quality...")
    
    chat_history = ChatHistory()
    chat_history.add_user_message(f"Please analyze this travel advice for completeness:\n\n{content}")
    
    async for response in reflection_agent.invoke(chat_history):
        try:
            reflection_result = json.loads(str(response.content))
            return reflection_result
        except json.JSONDecodeError:
            print(f"Failed to parse reflection response: {response.content}")
            return {"missing_topics": []}
    
    return {"missing_topics": []}

Create the main reflection workflow

In [ ]:
async def reflection_workflow(user_query: str, max_iterations: int = 3) -> str:
    """
    Main reflection workflow that iteratively improves content
    """
    print(f"Starting reflection workflow for: '{user_query}'\n")
    
    current_advice = ""
    additional_requirements = ""
    
    for iteration in range(max_iterations):
        print(f"=== Iteration {iteration + 1} ===")
        
        # Generate or refine travel advice
        current_advice = await get_travel_advice(user_query, additional_requirements)
        
        # Reflect on the content
        reflection = await reflect_on_content(current_advice)
        
        missing_topics = reflection.get("missing_topics", [])
        
        print(f"Missing topics: {missing_topics}")
        
        # If no topics are missing, we're done
        if not missing_topics:
            print("✅ Content is complete!")
            break
        
        # Prepare additional requirements for next iteration
        if iteration < max_iterations - 1:
            additional_requirements = f"Please extend your response with more information about: {', '.join(missing_topics)}"
            print(f"→ Requesting additional information about: {', '.join(missing_topics)}\n")
        else:
            print(f"⚠️ Maximum iterations reached. Some topics may still be missing: {missing_topics}")
    
    return current_advice

Test the reflection workflow

In [ ]:
# Test with a simple query
user_query = "Perth"
final_advice = await reflection_workflow(user_query)

print("\n" + "="*50)
print("FINAL TRAVEL ADVICE:")
print("="*50)
print(final_advice)

Test with a more complex query

In [ ]:
# Test with a more detailed query
complex_query = "I want to visit Rome for a week. What should I know?"
final_advice = await reflection_workflow(complex_query)

print("\n" + "="*50)
print("FINAL TRAVEL ADVICE FOR ROME:")
print("="*50)
print(final_advice)

## Detailed Reflection Analysis

Let's see what the reflection agent analyzes in detail:

In [ ]:
# Get initial advice
initial_advice = await get_travel_advice("Tokyo")
print("Initial Advice:")
print("-" * 30)
print(initial_advice)

# Get detailed reflection
reflection_analysis = await reflect_on_content(initial_advice)
print("\n\nReflection Analysis:")
print("-" * 30)
print(json.dumps(reflection_analysis, indent=2))

## Key Differences from LangChain Reflection

1. **Agent-Based Reflection**: Uses dedicated `ChatCompletionAgent` instances for different roles (advisor and reflector) rather than stateless functions.

2. **Simplified Workflow**: Instead of complex graph states and commands, uses a straightforward async workflow function that's easy to understand and modify.

3. **Flexible Iteration Control**: Easy to control the number of reflection iterations and customize the improvement process.

4. **Structured Analysis**: The reflection agent provides detailed JSON analysis of content completeness, making it easy to understand what's missing.

5. **Natural Language Coordination**: Agents communicate through natural language rather than structured state objects.

6. **Better Separation of Concerns**: Each agent has a specific role and can be independently configured and tested.

7. **Incremental Improvement**: Each iteration builds upon the previous result, maintaining context and continuity.

### Benefits:

- **Maintainability**: Simple workflow logic that's easy to debug and modify
- **Flexibility**: Easy to adjust reflection criteria or add new validation rules
- **Transparency**: Clear visibility into what the reflection agent is analyzing
- **Scalability**: Can easily add more specialized reflection agents for different criteria
- **Reusability**: Reflection agents can be reused across different workflows

### Use Cases:

- **Content Quality Assurance**: Ensuring responses meet specific quality criteria
- **Completeness Validation**: Verifying all required topics are covered
- **Iterative Refinement**: Progressively improving content through multiple rounds
- **Multi-criteria Evaluation**: Checking multiple aspects of generated content
- **Educational Content**: Ensuring learning materials cover all required topics"